**Tensorflow 2.1**

*   What is Image Classification?
*   What needs to be prepared to create an image classifier?
*   Prepare the dataset
*   Creating a pipeline for the input dataset
*   Building a Convolutional Neural Network Model
*   Using existing CNN Model Architecture
*   Compile the Model
*   Model Training
*   Model Save
*   Model Evaluation
*   Create Predict Function


# Mount Google Drive

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# cd into TensorFlow directory in Google Drive
%cd '/content/gdrive/My Drive/TensorFlow/'

# Preparing the Dataset

In this process, we want to divide the data into 3 parts which are train, test, and validation, with proportions 80% of train data, 10% of test data, and 10% of validation data

*   train (data for training)
*   validation (data for validating the training data)
*   test (data for testing the trained model)

In [ ]:
# extract dataset if needed (in zip format)
# !unzip handphone-photos.zip

In [ ]:
import os

mypath= 'handphone_photos/'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(mypath):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [ ]:
import pandas as pd

# input the variables that have been collected in the loop above into a dataframe to make the arrangement neatly
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

In [ ]:
# check data sample
df.head()

# Train test split

In [ ]:
# load library for train test split
from sklearn.model_selection import train_test_split

In [ ]:
# used variables in data split
X= df['path']
y= df['tag']

In [ ]:
# split first dataset become train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=300)

In [ ]:
# divide the test data into test and validation data
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=100)

In [ ]:
# combine into each dataframe
df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_te = pd.DataFrame({'path':X_test
              ,'tag':y_test
             ,'set':'test'})

df_val = pd.DataFrame({'path':X_val
              ,'tag':y_val
             ,'set':'validation'})

In [ ]:
print('train size', len(df_tr))
print('val size', len(df_te))
print('test size', len(df_val))

In [ ]:
# view the proportion of each set whether it is okay or if there is something need to change
df_all = df_tr.append([df_te,df_val]).reset_index(drop=1)\

print('===================================================== \n')
print(df_all.groupby(['set','tag']).size(),'\n')

print('===================================================== \n')

# check data sample
df_all.sample(3)

Tidy up to respective folders

In [ ]:
# menghapus folder dataset jika diperlukan
#!rm -rf dataset/

In [ ]:
import shutil
from tqdm.notebook import tqdm as tq

In [ ]:
datasource_path = "handphone_photos/"
dataset_path = "dataset/"

In [ ]:
for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

# Model Training

Initialize global variables for model training

In this section, we will determine how many epochs will be used for the training process, input dimension, batch_size, etc

In [ ]:
import tensorflow as tf

# Input parameters for network
dim = (150, 150) # value will be different based on used network architecture
channel = (3, )
input_shape = dim + channel

# Batch size
batch_size = 16

# Epoch
epoch = 10

The pipeline dataset is a command to extract data in the form of digital images from a folder into an array that can be read by tensorflow, the function used is 

```
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
```

This function is an Image data generator so that we generate image data from a file / folder that we created earlier. in this section we can determine what kind of generator / augmentation can be done.
For further reading please visit the following reference links:
- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
- https://keras.io/preprocessing/image/

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
First, we define how the data transformation will be used

This process will run:
* Rescaling data to 1/255
* 0.2 scale image shearing
* Zooming image with a range of 0.2
* and do a Horizontal flip

Reference: https://towardsdatascience.com/machinex-image-data-augmentation-using-keras-b459ef87cd22

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1. / 255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# Make the data flow
After defining the generator, then we define where the data source comes from. For example in this project we use data from a folder (dictionary) so that the code used is as follows:

In [ ]:
# binary = [1,0,0,0,0] [0,1,0,0,0] [0,0,1,0,0] [0,0,0,1,0] [0,0,0,0,1]
# categorical = 1,2,3,4,5

train_generator = train_datagen.flow_from_directory('dataset/train/',
                                                    target_size=dim,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = val_datagen.flow_from_directory('dataset/validation/',
                                                target_size=dim,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True)

test_generator = test_datagen.flow_from_directory('dataset/test/',
                                                  target_size=dim,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

num_class = test_generator.num_classes
labels = train_generator.class_indices.keys()

In [ ]:
print(labels)

In [ ]:
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None
                        , input_shape[0]
                        , input_shape[1]
                        , input_shape[2]]
                       ,[None, num_class])
    )
    return tf_generator

In [ ]:
train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

#(Optional)Creating Convolutional Neural Network Structures

We can skip this process if we want to use existing structures

In [ ]:
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class))
model.add(Activation('softmax'))

# Compile the model
print('Compiling Model.......')
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

# Using pre-build CNN structures (MobileNetV2)

In [ ]:
from tensorflow.keras.applications import MobileNetV2

# get base models
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet',
    classes=num_class,
)

In [ ]:
from tensorflow.keras import layers,Sequential
from tensorflow.keras.models import Model

In [ ]:
# adding custom layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1024, activation="relu")(x)

predictions = layers.Dense(num_class, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
# Compile the model
print('Compiling Model')
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Visualize the model
model_viz = tf.keras.utils.plot_model(model,
                          to_file='model.png',
                          show_shapes=True,
                          show_layer_names=True,
                          rankdir='TB',
                          expand_nested=True,
                          dpi=55)
model_viz

# Using pre-build CNN structures (EfficientNet)

In [ ]:
!pip install -U --pre efficientnet

In [ ]:
from efficientnet.tfkeras import EfficientNetB1

In [ ]:
# get base models
base_model = EfficientNetB1(
    input_shape=input_shape,
    include_top=False,
    weights='noisy-student',
    classes=num_class,
)

In [ ]:
from tensorflow.keras import layers,Sequential
from tensorflow.keras.models import Model

In [ ]:
#Adding custom layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1024, activation="relu")(x)

predictions = layers.Dense(num_class, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
# Compile the model
print('Compiling Model.......')
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Visualize the model

model_viz = tf.keras.utils.plot_model(model,
                          to_file='model.png',
                          show_shapes=True,
                          show_layer_names=True,
                          rankdir='TB',
                          expand_nested=True,
                          dpi=55)
model_viz

# Model Training

In [ ]:
EPOCH = 10

history = model.fit(x=train_data,
        steps_per_epoch=len(train_generator),
        epochs=EPOCH,
        validation_data=val_data,
        validation_steps=len(val_generator), 
        shuffle=True,
        verbose = 1)

In [ ]:
history.history['loss']

In [ ]:
history.history['accuracy']

# Plot the training

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# Plot history: MAE
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Plot history: MSE
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

#Save Model

In [ ]:
MODEL_BASE_PATH = "model"
PROJECT_NAME = "medium_project"
SAVE_MODEL_NAME = "model-effinetb7-161120.h5"
save_model_path = os.path.join(MODEL_BASE_PATH, PROJECT_NAME, SAVE_MODEL_NAME)

if os.path.exists(os.path.join(MODEL_BASE_PATH, PROJECT_NAME)) == False:
    os.makedirs(os.path.join(MODEL_BASE_PATH, PROJECT_NAME))
    
print('Saving Model At {}...'.format(save_model_path))
model.save(save_model_path,include_optimizer=False)

#Evaluate Model

In [ ]:
loss, acc = model.evaluate(test_data,steps=len(test_generator),verbose=0)
print('Accuracy on training data: {:.4f} \nLoss on training data: {:.4f}'.format(acc,loss),'\n')
 
loss, acc = model.evaluate(test_data,steps=len(test_generator),verbose=0)
print('Accuracy on test data: {:.4f} \nLoss on test data: {:.4f}'.format(acc,loss),'\n')

#Make a Prediction

In [ ]:
import requests
from io import BytesIO
from PIL import Image
import numpy as np

# Parameters
input_size = (150,150) # customizable

# Define Input Shape (3D)
channel = (3,)
input_shape = input_size + channel

# Define Labels
labels = ['cropped', 'full']

In [ ]:
def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr
    
def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

In [ ]:
#cd into the TensorFlow directory in your Google Drive
%cd '/content/gdrive/My Drive/TensorFlow'

# Load Model
from tensorflow.keras.models import load_model
MODEL_PATH = 'model/medium_project/model-effinetb7-161120.h5'
model = load_model(MODEL_PATH,compile=False)

In [ ]:
#cd into the TensorFlow directory in your Google Drive
%cd '/content/gdrive/My Drive'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img=['phone-pink.jpg', 'image.jpg', 'image-crop.jpg', 'testing-real (12).jpeg']

for image_path in img:
  print(image_path)
  # read image
  im = Image.open(image_path)
  X = preprocess(im,input_size)
  X = reshape([X])
  y = model.predict(X)

  plt.imshow(im)
  plt.show()

  print( labels[np.argmax(y)], np.max(y) )